In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import csv
import math
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import pytorch_lightning as pl
import glob

## First, we slice the train images into 31 x 31 pixels with the ground truth in the middle

In [2]:
def ndigit(n, x):
    x = str(x)
    while(len(x) < n):
        x = "0" + x
    return x

In [3]:
def load_data(res, files = 20):
    j = 0
    path = ["02", "train"]
    res = int((res-1)/2)
    
    for p in path:
        for f in range(files):
            image = np.load(f"images_{p}/images/image_{ndigit(3, f)}.npy")
            mask = np.load(f"masks_{p}/masks/mask_{ndigit(3, f)}.npy")
            image = np.reshape(image, (1024,1024,10))
            mask = np.reshape(mask, (1024,1024,1))

            # Add padding to every image (and mask) edge in case there are ground truths which are too close to an edge
            padded_image = np.pad(image, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')
            padded_mask = np.pad(mask, ((res+1, res+1), (res+1, res+1), (0,0)), mode='constant')

            # Extract ground truths
            ground_truths_pos = np.array(np.where(padded_mask != 0)).T
            
            # Slice and save patches around each ground truth
            for i in ground_truths_pos: 
                patch = (padded_image[i[0]-res : i[0]+res+1, i[1]-res : i[1]+res+1, :], padded_mask[i[0], i[1], 0])
                np.save(f"patches/train/patch_{p}_{ndigit(3, f)}_{ndigit(5, j)}.npy", np.array(patch, dtype="object"))                                 
                j += 1

In [4]:
# Check number of ground truths
# pos = 0
# for i in range(20):
#     mask = np.load(f"masks_02/masks/mask_{ndigit(3, i)}.npy")
#     ground_truths_pos = np.array(np.where(mask != 0)).T
#     pos = pos + len(ground_truths_pos)
# print(pos)

In [5]:
res = 15
#load_data(res)

## Then, we load the data and have a look

In [39]:
batch_size = 3

In [7]:
# Load patches
directory = 'patches/train'
file_paths = glob.glob(directory + '/*.npy')
trainset0 = [np.load(file_path, allow_pickle=True) for file_path in file_paths]
trainset = []
for pic in trainset0:
    trainset.append(pic)
len(trainset)

38863

In [8]:
trainset[0]

array([array([[[1828, 2589, 2589, ..., 3113, 3113, 4216],
               [4216, 3720, 3720, ..., 3177, 3177, 2852],
               [2852, 2577, 2577, ..., 2987, 2987, 3099],
               ...,
               [3296, 3180, 3180, ..., 2839, 2839, 2809],
               [2809, 3003, 3003, ..., 2901, 2901, 3862],
               [3862, 4495, 4495, ..., 3690, 3690, 2986]],

              [[2752, 3649, 3649, ..., 3318, 3318, 2660],
               [2660, 2365, 2365, ..., 1897, 1897, 2049],
               [2049, 2513, 2513, ..., 2798, 2798, 2941],
               ...,
               [3058, 3025, 3025, ..., 2702, 2702, 2982],
               [2982, 3786, 3786, ..., 3334, 3334, 3035],
               [3035, 3121, 3121, ..., 2796, 2796, 3001]],

              [[4135, 3565, 3565, ..., 2924, 2924, 3659],
               [3659, 3214, 3214, ..., 2080, 2080, 2523],
               [2523, 2724, 2724, ..., 2840, 2840, 2813],
               ...,
               [2994, 3207, 3207, ..., 3519, 3519, 2896],
        

In [9]:
def enrich_channels(trainset, veggie, moisture):
    #trainset[pic_no][0][h][w][channel] -> pixel value
    print(f"Shape vorher: Liste mit ({res},{res},10) Bildern")
    counter = 0
    trainset = trainset.copy()  # Make a copy of the trainset

    if veggie:
        pic_no = 0
        for pic in trainset:
            counter += 1
            pixel_values = pic[0]
            channel8 = pixel_values[:, :, 7]
            channel4 = pixel_values[:, :, 3]
            channels = pic[0].shape[2]
            width = pic[0].shape[0]
            height = pic[0].shape[1]

            vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(channel8, channel4))
            trainset_transformed = np.concatenate((trainset[pic_no][0], vegetation_array[:, :, np.newaxis]), axis=2)
            trainset[pic_no] = (trainset_transformed, trainset[pic_no][1])
            pic_no += 1

        print("Added Vegetation (B8-B4)/(B8+B4)")

    if moisture:
        pic_no = 0
        for pic in trainset:
            pixel_values = pic[0]
            channel8a = pixel_values[:, :, 7]
            channel11 = pixel_values[:, :, 8]
            channels = pic[0].shape[2]
            width = pic[0].shape[0]
            height = pic[0].shape[1]

            moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(channel8a, channel11))
            trainset_transformed = np.concatenate((trainset[pic_no][0], moisture_array[:, :, np.newaxis]), axis=2)
            trainset[pic_no] = (trainset_transformed, trainset[pic_no][1])
            pic_no += 1

        print("Added Moisture (B8A-B11)/(B8A+B11)")
        
    print("shape nachher", trainset[0][0].shape)  # Print the shape of the first item

    return trainset

In [10]:
trainset_enriched = enrich_channels(trainset, True, True)
print(trainset_enriched[0])
#len(trainset_enriched)
#trainset[pic_no][0][h][w][channel] -> pixel value
#trainset[pic_no][1] -> Ground truth 

Shape vorher: Liste mit (15,15,10) Bildern


/var/folders/8w/vxj71qs163b3nd515vcdbpbc0000gn/T/ipykernel_23227/950853731.py:18: RuntimeWarning: invalid value encountered in divide
  vegetation_array = np.divide((np.subtract(channel8, channel4)), np.add(channel8, channel4))


Added Vegetation (B8-B4)/(B8+B4)


/var/folders/8w/vxj71qs163b3nd515vcdbpbc0000gn/T/ipykernel_23227/950853731.py:35: RuntimeWarning: invalid value encountered in divide
  moisture_array = np.divide((np.subtract(channel8a, channel11)), np.add(channel8a, channel11))


Added Moisture (B8A-B11)/(B8A+B11)
shape nachher (15, 15, 12)
(array([[[ 1.82800000e+03,  2.58900000e+03,  2.58900000e+03, ...,
          4.21600000e+03,  2.33399080e-02,  0.00000000e+00],
        [ 4.21600000e+03,  3.72000000e+03,  3.72000000e+03, ...,
          2.85200000e+03,  1.39322216e-01,  0.00000000e+00],
        [ 2.85200000e+03,  2.57700000e+03,  2.57700000e+03, ...,
          3.09900000e+03, -3.50291910e-03,  0.00000000e+00],
        ...,
        [ 3.29600000e+03,  3.18000000e+03,  3.18000000e+03, ...,
          2.80900000e+03, -6.31908926e-02,  0.00000000e+00],
        [ 2.80900000e+03,  3.00300000e+03,  3.00300000e+03, ...,
          3.86200000e+03, -3.18705156e-02,  0.00000000e+00],
        [ 3.86200000e+03,  4.49500000e+03,  4.49500000e+03, ...,
          2.98600000e+03, -1.01315149e-01,  0.00000000e+00]],

       [[ 2.75200000e+03,  3.64900000e+03,  3.64900000e+03, ...,
          2.66000000e+03, -4.88748746e-02,  0.00000000e+00],
        [ 2.66000000e+03,  2.36500000e+0

In [11]:
X,y = trainset_enriched[0]
X.shape

(15, 15, 12)

In [12]:
import albumentations as A
elastic_transform = A.ElasticTransform(alpha=50.0, sigma=3.0)

class CustomDataset(Dataset):
    def __init__(self, trainset, transform):
        self.trainset = trainset
        self.transform = transform
        #self.augmentations = augmentations

    def __len__(self):
        return len(self.trainset)

    def __getitem__(self, index):
        data, target = self.trainset[index]

        # apply each transformation jointly to each input
        if self.transform:
            data = self.transform(data)

        # apply each augmentation separately to each input
       # if self.augmentations:
       #     for augmentation in self.augmentations:
       #         data = augmentation(data)


        return data, target

#def to_float64(image):
 #   return image.to(torch.float64)

def normalize(image):
    image = image/3000
    return torch.where(image > 1, 1, image)

class CustomTransform:
    def __init__(self):
        self.mean = (0.5,) * 12
        self.std = (0.5,) * 12

    def __call__(self, image):
        #image = to_float64(image)
        image = normalize(image)
        image = transforms.Normalize(self.mean, self.std)(image)
        return image

transform = transforms.Compose([
    transforms.ToTensor(),
    CustomTransform()
])


augmentations = [
     transforms.RandomRotation(360),
     transforms.RandomAffine(degrees=0, translate=(0.5,0.5)), # shift in both directions along 0.5 * height on y-axis and 0.5 * width on x-axis
     transforms.RandomAffine(0, scale=(10,45)), # scale in range 10 <= scale <= 45
     transforms.RandomAffine(0, shear=[10,30,10,30]), # shear on x- and y-axis between (10,30) 
     #elastic_transform # displaces pixels
]

In [13]:
# Create the custom dataset
trainset_transformed = CustomDataset(trainset_enriched, transform=transform)
trainset_transformed[0][0]
#on the fly augmentation during training, hence no additional pictures in trainset 
#len(trainset_transformed)

tensor([[[ 0.2187,  1.0000,  0.9013,  ...,  1.0000,  0.8727,  1.0000],
         [ 0.8347,  0.7733,  0.3660,  ...,  1.0000,  0.9880,  1.0000],
         [ 1.0000,  1.0000,  0.6820,  ...,  0.9960,  0.9307,  1.0000],
         ...,
         [ 0.8507,  0.9367,  0.9353,  ...,  1.0000,  1.0000,  0.5560],
         [ 0.8687,  0.9113,  0.9373,  ...,  1.0000,  0.9693,  1.0000],
         [ 0.7873,  0.9367,  0.8867,  ...,  0.9333,  0.9353,  1.0000]],

        [[ 0.7260,  1.0000,  0.7180,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  0.5767,  0.6753,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  0.8160,  ...,  1.0000,  0.9913,  1.0000],
         ...,
         [ 0.8667,  0.9333,  0.9033,  ...,  0.9700,  1.0000,  0.6360],
         [ 0.9413,  0.9153,  0.9160,  ...,  1.0000,  0.9553,  1.0000],
         [ 0.8073,  0.9013,  0.9107,  ...,  0.9933,  1.0000,  1.0000]],

        [[ 0.7260,  1.0000,  0.7180,  ...,  1.0000,  1.0000,  1.0000],
         [ 1.0000,  0.5767,  0.6753,  ...,  1

In [14]:
#trainset_transformed[0]

In [40]:
# Calculate the sizes of the training set and validation set
train_size = int(0.8 * len(trainset_transformed))
val_size = len(trainset_transformed) - train_size

# Split trainset into trainset and valset
trainset_load, valset_load = random_split(trainset_transformed, [train_size, val_size])
print(len(trainset_load), len(valset_load))

# Create data loaders for the training set and validation set
trainloader = DataLoader(trainset_load, batch_size=batch_size, shuffle=True)
validloader = DataLoader(valset_load, batch_size=batch_size, shuffle=False)


31090 7773


In [24]:
limit = 0
for batch in trainloader:

    while(limit <1):
        print(batch)
    #    print(x.shape)
    #    print(x.size)
    #    print(y.shape)
    #    print(y)
        limit += 1

[tensor([[[[-0.6920, -0.6807, -0.6900,  ..., -0.7220, -0.6773, -0.6700],
          [-0.6633, -0.6887, -0.6913,  ..., -0.6713, -0.5960, -0.6780],
          [-0.6933, -0.6953, -0.6913,  ..., -0.7027, -0.6933, -0.6880],
          ...,
          [-0.6800, -0.6787, -0.6873,  ..., -0.6473, -0.6573, -0.3120],
          [-0.6613, -0.6833, -0.6813,  ..., -0.6693, -0.2347, -0.2787],
          [-0.6320, -0.6260, -0.7140,  ..., -0.6613, -0.3840, -0.3640]],

         [[-0.6793, -0.6653, -0.6933,  ..., -0.7253, -0.7020, -0.6587],
          [-0.6713, -0.6947, -0.6887,  ..., -0.6747, -0.6673, -0.6993],
          [-0.7007, -0.7080, -0.6847,  ..., -0.6780, -0.6980, -0.6873],
          ...,
          [-0.6613, -0.6780, -0.7000,  ..., -0.6413, -0.6493, -0.3133],
          [-0.6733, -0.6767, -0.6813,  ..., -0.6800, -0.2107, -0.2907],
          [-0.6440, -0.6573, -0.7313,  ..., -0.6800, -0.3747, -0.3787]],

         [[-0.6653, -0.6747, -0.6853,  ..., -0.7327, -0.6993, -0.6333],
          [-0.6713, -0.7047, 

## Next, we define the model and train it

In [49]:
class MyCNNModel(pl.LightningModule): # New! def init(self, layers, lr=0.01, classes=None): super().init() # <- Very important! self.lr = lr self.classes = classes ## Build model self.layers = nn.Sequential(layers) # Create a sequential model

    def __init__(self, *layers, classes=None):
        super().__init__()

        self.lr = 0.01  # Assign the learning rate here
        self.classes = classes

        self.layers = nn.Sequential(*layers)  # Create a sequential model
        
    def forward(self, X):
        return self.layers(X)

    def predict(self, X):
        with torch.no_grad():
            y_hat = self(X).argmax(1)
        if self.classes is not None:
            y_hat = [self.classes[i] for i in y_hat]
        return y_hat

    def training_step(self, batch, batch_idx, log_prefix='train'):
        X, y = batch
        y_hat = self(X)
        if(y_hat == None):
            return 0
        loss_fn = nn.MSELoss()
        loss = loss_fn(y_hat, y)
        #print(y_hat, "yhat")
        
        print(y_hat.size)
        print(y_hat.shape)
        self.log(f"{log_prefix}_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            return self.training_step(batch, batch_idx, log_prefix='valid')

    def configure_optimizers(self):
        # Adam with Weight Decay
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)

        # Simplest scheduler is ReduceLROnPlateau. This scheduler reduces the learning rate by 0.1
        # if the val_loss has not decreased within the last 10 epochs.
        scheduler = {
            # REQUIRED: The scheduler instance
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=10, verbose=True),
            # The unit of the scheduler's step size, could also be 'step'.
            # 'epoch' updates the scheduler on epoch end whereas 'step'
            # updates it after a optimizer update.
            "interval": "epoch",
            # How many epochs/steps should pass between calls to
            # `scheduler.step()`. 1 corresponds to updating the learning
            # rate after every epoch/step.
            "frequency": 1,
            # Metric to to monitor for schedulers like `ReduceLROnPlateau`
            "monitor": "valid_loss",
            # If set to `True`, will enforce that the value specified 'monitor'
            # is available when the scheduler is updated, thus stopping
            # training if not found. If set to `False`, it will only produce a warning
            "strict": True,
            # If using the `LearningRateMonitor` callback to monitor the
            # learning rate progress, this keyword can be used to specify
            # a custom logged name
            "name": None,
        }
        return {"optimizer": optimizer, 'lr_scheduler': scheduler}

## Implement model

In [27]:
# Implements entry to SepConv2d, see Lang et al. (2019), p. 6
class MyEntryLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.out_channels = out_channels

        self.proj_out = nn.Conv2d(in_channels, out_channels[len(out_channels)-1], (1,1))

        self.entry_blocks = nn.ModuleList()
        for i in range(len(out_channels)):
            self.entry_blocks.append(nn.Sequential(
                nn.Conv2d(in_channels, out_channels[i], (1, 1)),
                nn.BatchNorm2d(out_channels[i]),
                nn.ReLU(inplace = True)
            ))
            in_channels = out_channels[i]  # Update in_channels for next iteration

    def forward(self, x):
        x_entry = x
        for i in range(len(self.out_channels)):
            x_entry = self.entry_blocks[i](x_entry)
        x = self.proj_out(x)
        return (x + x_entry)

In [28]:
# Implements SepConv2D
class MySepConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel, **kwargs):
        super().__init__()
        if in_channels == out_channels:
            self.proj_out = nn.Identity()
        else:
            self.proj_out = nn.Conv2d(in_channels, out_channels, (1,1), **kwargs)

        self.sep_conv_block = nn.Sequential(
            nn.ReLU(inplace = True),
            nn.Conv2d(in_channels, in_channels, kernel, groups=in_channels, **kwargs), # depthwise SepConv
            nn.Conv2d(in_channels, out_channels, (1,1), **kwargs), # pointwise SepConv
            nn.BatchNorm2d(out_channels)
        )
    
    def forward(self, x):
        x_sep_conv = self.sep_conv_block(x)
        x_sep_conv_2 = self.sep_conv_block(x_sep_conv) # performs second SepConv, see Lang et al. (2019), p. 6
        x = self.proj_out(x)
        return (x + x_sep_conv_2) # adds original input and sep_conv_2 output

In [50]:
tree_model = MyCNNModel(
    #MyEntryLayer(12, [128, 256]), # increase number of channels to 512
    nn.Conv2d(12, 256, (3,3), padding='same'),
    #MySepConvLayer(256, 256, (3,3), padding='same'),
    #MySepConvLayer(256, 256, (3,3), padding='same'),
    nn.AdaptiveMaxPool2d(1),
    nn.Flatten(1),
    nn.Linear(256, 1)
)

In [51]:
# New, we need a trainer class
from pytorch_lightning.callbacks import RichProgressBar, RichModelSummary
trainer1 = pl.Trainer(devices=1, accelerator="cpu", precision='64', max_epochs=1,
                      callbacks=[RichProgressBar(refresh_rate=50),
                                 RichModelSummary(3),
                                ])

Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.rich_model_summary.RichModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer1.fit(tree_model, trainloader, validloader)

┏━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name     ┃ Type              ┃ Params ┃
┡━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ layers   │ Sequential        │ 28.2 K │
│ 1 │ layers.0 │ Conv2d            │ 27.9 K │
│ 2 │ layers.1 │ AdaptiveMaxPool2d │      0 │
│ 3 │ layers.2 │ Flatten           │      0 │
│ 4 │ layers.3 │ Linear            │    257 │
└───┴──────────┴───────────────────┴────────┘

Trainable params: 28.2 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 28.2 K                                                                                               
Total estimated model params size (MB): 0

Output()

<built-in method size of Tensor object at 0x7fe13e96f9c0>

torch.Size([3, 1])

...

torch.Size([3, 1])

<built-in method size of Tensor object at 0x7fe13e96ede0>

torch.Size([3, 1])

<built-in method size of Tensor object at 0x7fe13e96ede0>

torch.Size([3, 1])

In [ ]:
tree_model.eval()
tree_model = tree_model.float()
batch = next(iter(trainloader))
inputs = batch[0]
inputs = inputs.float()

print(inputs.shape)


with torch.no_grad():
    predictions = tree_model(inputs)


print("Predictions:", predictions)
# expected 128 pictures, each with 15 * 15 predictions for height ! 